In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import matplotlib.pyplot as plt

tfd = tfp.distributions

/Users/priyarajni/Desktop/Pythoncode/MSAI-630-A02/venvMLP/lib/python3.13/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
tf.random.set_seed(42)
np.random.seed(42)

In [3]:
(x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()

In [4]:
x_train = x_train[..., None]
x_test  = x_test[..., None]

In [5]:
x_train = tf.convert_to_tensor(x_train, dtype=tf.int32)
x_test  = tf.convert_to_tensor(x_test,  dtype=tf.int32)

In [6]:
IMAGE_SIZE = 28
CHANNELS = 1
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)

BATCH_SIZE = 128
train_ds = (tf.data.Dataset.from_tensor_slices(x_train)
            .shuffle(10000)
            .batch(BATCH_SIZE)
            .prefetch(tf.data.AUTOTUNE))

In [7]:
N_COMPONENTS = 5  # mixture components (try 5 or 10)

dist = tfd.PixelCNN(
    image_shape=IMAGE_SHAPE,
    num_resnet=5,          # model capacity
    num_hierarchies=2,
    num_filters=64,
    num_logistic_mix=N_COMPONENTS,  # <-- mixture distribution output
    dropout_p=0.3
)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

@tf.function
def train_step(x):
    # x comes in as int32 [0,255]
    x = tf.cast(x, tf.float32)   # <-- FIX

    with tf.GradientTape() as tape:
        log_prob = dist.log_prob(x)
        loss = -tf.reduce_mean(log_prob)

    grads = tape.gradient(loss, dist.trainable_variables)
    optimizer.apply_gradients(zip(grads, dist.trainable_variables))
    return loss

In [9]:
@tf.function
def eval_nll(x):
    return -tf.reduce_mean(dist.log_prob(x))

In [10]:
EPOCHS = 20
train_losses = []

print("Training PixelCNN...")
for epoch in range(EPOCHS):
    batch_losses = []
    for batch in train_ds:
        l = train_step(batch)
        batch_losses.append(l)
    epoch_loss = tf.reduce_mean(batch_losses)
    train_losses.append(float(epoch_loss.numpy()))
    print(f"Epoch {epoch+1:02d}/{EPOCHS}  Train NLL: {train_losses[-1]:.3f}")

Training PixelCNN...


TypeError: in user code:

    File "/var/folders/_b/_ly21q_90t35b34bgq1n5jhw0000gn/T/ipykernel_69263/2307986568.py", line 7, in train_step  *
        log_prob = dist.log_prob(x)           # (batch,)
    File "/Users/priyarajni/Desktop/Pythoncode/MSAI-630-A02/venvMLP/lib/python3.13/site-packages/tensorflow_probability/python/distributions/distribution.py", line 1287, in log_prob  **
        return self._call_log_prob(value, name, **kwargs)
    File "/Users/priyarajni/Desktop/Pythoncode/MSAI-630-A02/venvMLP/lib/python3.13/site-packages/tensorflow_probability/python/distributions/distribution.py", line 1269, in _call_log_prob
        return self._log_prob(value, **kwargs)
    File "/Users/priyarajni/Desktop/Pythoncode/MSAI-630-A02/venvMLP/lib/python3.13/site-packages/tensorflow_probability/python/distributions/pixel_cnn.py", line 423, in _log_prob
        transformed_value = (2. * (value - self._low) /

    TypeError: Input 'y' of 'Sub' Op has type float32 that does not match type int32 o

In [ ]:
plt.figure()
plt.plot(train_losses)
plt.xlabel("Epoch")
plt.ylabel("Negative Log-Likelihood (NLL)")
plt.title("PixelCNN Training Loss Curve")
plt.show()

In [ ]:
NUM_SAMPLES = 16
samples = dist.sample(NUM_SAMPLES)  # (16, 28, 28, 1), int values
samples_np = samples.numpy().astype(np.uint8)

plt.figure(figsize=(6, 6))
for i in range(NUM_SAMPLES):
    plt.subplot(4, 4, i + 1)
    plt.imshow(samples_np[i, ..., 0], cmap="gray")
    plt.axis("off")
plt.suptitle("PixelCNN Generated Samples (tfp.distributions.PixelCNN.sample)")
plt.show()

In [ ]:
def pixel_accuracy_on_batch(x_int):
    return None

test_batch = x_test[:1024]
test_nll = float(eval_nll(test_batch).numpy())
print(f"\nTest NLL (lower is better): {test_nll:.3f}")